# Предварительная настройка среды разработки

## Глобальные параметры

Глобальные параметры работы блокнота для упрощения управления процессом обучения:

In [1]:
dataset_name = "mozilla-foundation/common_voice_11_0"                # Название датасета
dataset_lang = "ru"                                                 # Язык датасета

model_name = "whisper-tiny"
model_task = "transcribe"

tokenizer_language = "Russian"


# Автоматически формируемые параметры
reference_model = "openai/" + model_name
result_model_name = model_name + "-fine_tuned-ru"

# DEBUG
print(dataset_name, dataset_lang)
print(tokenizer_language, model_task)
print(reference_model)
print(result_model_name)

mozilla-foundation/common_voice_2_0 ru
Russian transcribe
openai/whisper-tiny
whisper-tiny-fine_tuned-ru


## Установка необходимых библиотек

### Локальная среда

In [2]:
# Установка необходимых библиотек
# !conda install -y -c conda-forge transformers
# !conda install -y -c conda-forge datasets
# !conda install -y -c conda-forge librosa 
# !conda install -y -c conda-forge ffmpeg 
# !conda install -y -c conda-forge huggingface_hub 
# !conda install -y -c conda-forge tensorboard
# !conda install -y -c conda-forge git-lfs
# !pip install --no-input evaluate
# !pip install --no-input jiwer
# !pip install --no-input gradio
# !pip install huggingface_hub["cli"]
print("Library installation complete!")


print("Local environment setup complete.")

Library installation complete!
Local environment setup complete.


### Google Colab

In [ ]:
# Установка необходимых библиотек
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

# Подключим Google Drive
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

# Путь к папке для хранения датасета
dataset_path = "/content/drive/MyDrive/MY_WORK/Whisper_Train/DATA"  

### Kaggle

In [ ]:
# Путь к папке для хранения датасета
dataset_path = "/content/drive/MyDrive/MY_WORK/Whisper_Train/DATA"  

## Проверка доступности GPU

Чтобы получить GPU, нажмите _Runtime_ -> _Change runtime type_, затем измените _Hardware accelerator_ с _None_ на _GPU_.

Мы можем проверить, что нам назначен GPU, и просмотреть его характеристики:

In [3]:
!nvidia-smi

Sat Jan 14 20:57:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.92.01    Driver Version: 528.02       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P8     6W /  74W |      9MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Подключение библиотек

In [4]:
# Вспомогательные библиотеки
import os
import subprocess
import os
import multiprocessing
import warnings
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# HuggingFace
from huggingface_hub import notebook_login
from datasets import load_dataset, DatasetDict
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from datasets import Audio
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

# Evaluate
import evaluate

# PyTorch
import torch

print("Required libraries/modules initialization complete!")

Required libraries/modules initialization complete!


## Настройка окружения

In [5]:
# Определим количество ядер CPU, это влияет на количество рабочих процессов которые мы можем запустить одновременно
system_num_workers = multiprocessing.cpu_count()
print("The number of workers processes is set at", system_num_workers)

# Отключение назойлевых предупреждений
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Аутентификация ноутбука в HuggingFace HUB
# notebook_login()

print("Environment setup completed!")

The number of workers processes is set at 16
Environment setup completed!


# Тонкая настройка Whisper для многоязыкового ASR с помощью библиотеки 🤗 Transformers

В этом блокноте мы представляем пошаговое руководство по тонкой настройке Whisper для любого многоязыкового набора данных ASR с помощью библиотеки Hugging Face 🤗 Transformers. Это более "практическая" версия сопутствующего [сообщения в блоге](https://huggingface.co/blog/fine-tune-whisper). Для более подробного объяснения Whisper, набора данных Common Voice и теории, лежащей в основе тонкой настройки, читателю рекомендуется обратиться к статье в блоге.

## Введение

Whisper - это предварительно обученная модель для автоматического распознавания речи (ASR) опубликованная в [сентябре 2022](https://openai.com/blog/whisper/) авторами Алеком Рэдфордом и другими из OpenAI. В отличие от многих своих предшественников, таких как. 
[Wav2Vec 2.0](https://arxiv.org/abs/2006.11477), которые предварительно обученны на неразмеченных аудиоданных, Whisper предварительно обучен на огромном количестве **размеченных** транскрибированных аудио данных, 680 000 часов, если быть точным. Это на порядок больше данных, чем использованные для обучения Wav2Vec 2.0 (60 000 часов) неразмеченных аудиоданных. Более того, 117 000 часов из этих данных использованных для предварительного обучения - это многоязыковые данные ASR. Это позволяет получить контрольные точки которые могут быть применены к более чем 96 языкам, многие из которых считаются _низкоресурсными_.

При масштабировании на 680 000 часов размеченных данных предварительного обучения, модели Whisper демонстрируют высокую способность к обобщению для многих наборов данных и областей (доменов). Предварительно обученные контрольные точки достигают результатов, конкурентоспособных с современными 
ASR системами, с коэффициентом ошибок в словах (WER) около 3% на подмножестве чистых тестов LibriSpeech ASR и новым передовым результатом на TED-LIUM с 4,7% (смотрите таблицу 8 из [Whisper paper](https://cdn.openai.com/papers/whisper.pdf)). 

Обширные знания о многоязычном ASR, полученные Whisper во время предварительного обучения могут быть использованы для других языков с низким уровнем ресурсов; посредством тонкой настройки предварительно обученные контрольные точки могут быть адаптированы для конкретных наборов данных и языков для дальнейшего улучшения результатов. Мы покажем, как можно точно настроить Whisper для языков с ограниченными ресурсами в этом блокноте.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/whisper_architecture.svg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Рисунок 1:</b> модель Whisper. Архитектура соответствует стандартной модели кодера-декодера на основе трансформатора. Log-Mel спектрограмма подается на вход кодера. Последние скрытые состояния поступают в декодер через механизмы перекрестного внимания. Декодер авторегрессионно предсказывает текстовые токены, совместно обусловленные скрытыми состояниями энкодера и ранее предсказанными токенами. Источник рисунка: 
<a href="https://openai.com/blog/whisper/">блог OpenAI Whisper</a>.</figcaption>
</figure>

Контрольные точки Whisper представлены в пяти конфигурациях с различными размерами моделей. Самые маленькие четыре модели обучаются либо только на английском, либо на многоязычных данных. Самая большая контрольная точка - только многоязычная. 
Все девять предварительно обученных контрольных точек доступны на сайте [Hugging Face Hub](https://huggingface.co/models?search=openai/whisper). Контрольные точки сведены в следующую таблицу со ссылками на модели в хабе:

| Размер   | Количество слоёв | Ширина | Количество голов | Параметры | Только англоязычная модель                                         | Многоязычная модель                                      |
|--------|--------|-------|-------|------------|------------------------------------------------------|---------------------------------------------------|
| tiny   | 4      | 384   | 6     | 39 M       | [✓](https://huggingface.co/openai/whisper-tiny.en)   | [✓](https://huggingface.co/openai/whisper-tiny.)  |
| base   | 6      | 512   | 8     | 74 M       | [✓](https://huggingface.co/openai/whisper-base.en)   | [✓](https://huggingface.co/openai/whisper-base)   |
| small  | 12     | 768   | 12    | 244 M      | [✓](https://huggingface.co/openai/whisper-small.en)  | [✓](https://huggingface.co/openai/whisper-small)  |
| medium | 24     | 1024  | 16    | 769 M      | [✓](https://huggingface.co/openai/whisper-medium.en) | [✓](https://huggingface.co/openai/whisper-medium) |
| large  | 32     | 1280  | 20    | 1550 M     | x                                                    | [✓](https://huggingface.co/openai/whisper-large)  |

В демонстрационных целях мы доработаем контрольную точку многоязычной версии [``малой модели``](https://huggingface.co/openai/whisper-small) с 244M параметрами (~= 1GB). Что касается наших данных, мы обучим и оценим нашу систему на языке с низким уровнем ресурсов, взятом из набора данных [Common Voice](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Мы покажем, что всего за 8 часов тонкой настройки данных мы можем добиться высоких результатов на этом языке.

------------------------------------------------------------------------

${}^1$ - Название Whisper происходит от аббревиатуры "WSPSR", которая расшифровывается как "Web-scale Supervised Pre-training for Speech Recognition".

## Загрузка датасета

Используя 🤗 Datasets, загрузка и подготовка данных чрезвычайно проста. 

Мы можем загрузить и подготовить сплиты Common Voice всего одной строкой кода. 

Во-первых, убедитесь, что вы приняли условия использования на Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). После принятия условий вы получите полный доступ к набору данных и сможете загружать данные локально.

Объединим `train` и `validation` части набора данных, чтобы получить как можно больше данных для обучения. Будем использовать
данных `test` в качестве нашего тестового набора.

> ВАЖНО!
> Для удобства хранения данных датасета, я ранее определил путь к папке в переменной `DATA_PATH`. Это позволит хранить кеш данных в известном заранее месте и в дальшнейшем удалить ненужные данные. Это позволит быстро освободить место на HDD.

In [6]:
# Создадим словарь для хранения датасета
common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, dataset_lang, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset(dataset_name, dataset_lang, split="test", use_auth_token=True)

print(common_voice)
print("Data download successfully completed!")

Found cached dataset common_voice_2_0 (/home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079)
Found cached dataset common_voice_2_0 (/home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 3898
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1561
    })
})
Data download successfully completed!


Большинство наборов данных ASR предоставляют только входные образцы аудио (`audio`) и соответствующий транскрибированный текст (`sentence`).  Common Voice содержит дополнительную метаданные, такие как `accent` и `locale`, которые мы можем игнорировать для ASR.

Оставив блокнот максимально общим, мы рассматриваем только входной звук и транскрибированный текст для тонкой настройки, отбрасывая дополнительную информацию метаданных:

In [7]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3898
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1561
    })
})


## Подготовка экстрактора признаков, токенизатора и данных

Конвейер ASR можно разделить на три этапа: 
1) экстрактор признаков, который предварительно обрабатывает необработанные аудиоданные
2) Модель, которая выполняет сопоставление последовательности с последовательностью 
3) Токенизатор, который постобрабатывает выводы модели в текстовый формат.

Модель Whisper из библиотеки 🤗 Transformers имеет ассоциированный экстрактор признаков и токенизатор, называемые [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
и [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer) 
соответственно.

Рассмотрим детальнее настройки экстрактора и токенизатора по очереди!

### Загрузка WhisperFeatureExtractor

Экстрактор признаков Whisper выполняет две операции:
1. Разбивка / усечение аудиоданных до 30 секунд: все аудиоданные короче 30 секунд разбиваются на молчание (нули), а те, что длиннее 30 секунд, усекаются до 30 секунд.
2. Преобразует входные аудио в _log-Mel спектрограммы_ входных признаков, визуальное представление аудио и форму входных данных, ожидаемых моделью Whisper.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Рисунок 2:</b> Преобразование дискретизированного аудио массива в log-Mel спектрограмму.
Слева: дискретизированный одномерный аудиосигнал. Справа: соответствующая log-Mel спектрограмма. Источник рисунка:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

Мы загрузим экстрактор признаков из предварительно обученной контрольной точки со параметрами по умолчанию:

In [8]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(reference_model)

print("The feature extractor is loaded successfully.")

The feature extractor is loaded successfully.


### Загрузка WhisperTokenizer

Модель Whisper выдает последовательность идентификаторов _токенов_. Токенизатор сопоставляет каждый из этих идентификаторов токенов с соответствующей текстовой строкой. Для русского языка мы можем загрузить предварительно обученный токенизатор и использовать его для тонкой настройки 
с указанием целевого языка и задачи. Эти аргументы сообщают токенизатору о том, что следует задавать префикс токена языка и задачи в начале последовательности меток:

In [9]:
tokenizer = WhisperTokenizer.from_pretrained(reference_model,
                                             language = tokenizer_language,
                                             task = model_task)

print("Tokenizer loaded successfully.")

Tokenizer loaded successfully.


### Комбинирование для создания WhisperProcessor

Чтобы упростить использование экстрактора признаков и токенизатора, мы можем _обернуть_ их в один класс `WhisperProcessor`. Этот объект наследуется от `WhisperFeatureExtractor` и `WhisperProcessor`, и может использоваться для обработки входных аудиоданных и 
получения предсказаний модели по мере необходимости. При этом во время обучения нам нужно отслеживать только два объекта: `processor` и `model`:

In [10]:
processor = WhisperProcessor.from_pretrained(reference_model, 
                                             language = tokenizer_language,
                                             task = model_task)

print("Processor loaded successfully.")

Processor loaded successfully.


### Подготовка данных

Давайте распечатаем первый образец набора данных Common Voice, чтобы посмотреть в какой форме находятся данные:

In [11]:
print(common_voice["train"][0])

{'audio': {'path': '/home/artyom/.cache/huggingface/datasets/downloads/extracted/07c0c3b043e9d8c799510d1ed4b87e6d3a2e7c9716d6dcf311736070b42c4061/clips/001b11fa8d4b0277e41ab82671346ec1b8851e6ed7162c12952a45c9ffc3d8a61b333e92188ebfc6d61b128e1967fb0074b4d22208dc32f41a4f4a23e6ec1f00.mp3', 'array': array([ 0.0000000e+00,  4.5755869e-13,  8.2663303e-13, ...,
       -1.3604904e-07, -2.0340853e-05, -7.7015411e-06], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'Да ты уже просто с катушек съехал!'}


Поскольку наш входной звук сэмплирован с частотой 48 кГц (`'...sampling_rate': 48000...`), нам нужно _уменьшить_ его до частоту дискретизации до 16 кГц перед тем, как передать его в экстрактор признаков Whisper, 16 кГц - это частота дискретизации, ожидаемая моделью Whisper. Мы установим для аудио входов правильную частоту дискретизации с помощью метода [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column). Эта операция не изменяет звук "на месте", а скорее дает сигнализирует `datasets` передискретизировать аудиосэмплы _на лету_ при при первой загрузке данных:

In [12]:
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print("Audio sample rate set successfully.")

Audio sample rate set successfully.


Повторная загрузка первого аудиообразца из набора данных Common Voice приведет к его передискретизации до нужной частоты дискретизации:

In [13]:
print(common_voice["train"][0])

{'audio': {'path': '/home/artyom/.cache/huggingface/datasets/downloads/extracted/07c0c3b043e9d8c799510d1ed4b87e6d3a2e7c9716d6dcf311736070b42c4061/clips/001b11fa8d4b0277e41ab82671346ec1b8851e6ed7162c12952a45c9ffc3d8a61b333e92188ebfc6d61b128e1967fb0074b4d22208dc32f41a4f4a23e6ec1f00.mp3', 'array': array([ 3.0140032e-13,  7.2592689e-15, -1.1101574e-12, ...,
       -1.6450653e-05,  2.4153996e-07, -1.7190577e-06], dtype=float32), 'sampling_rate': 16000}, 'sentence': 'Да ты уже просто с катушек съехал!'}


Теперь мы можем написать функцию для подготовки наших данных к работе с моделью:
1. Мы загружаем и передискретизируем аудиоданные, вызывая `batch["audio"]`. Как объяснялось выше, 🤗 Datasets выполняет все необходимые операции по передискретизации на лету.
2. Мы используем экстрактор признаков для вычисления входных признаков спектрограммы log-Mel из нашего одномерного аудио массива.
3. Мы кодируем транскрипции в идентификаторы меток с помощью токенизатора.

In [14]:
def prepare_dataset(batch):
    # Загрузим и передискретизировать аудиоданные с 48 до 16 кГц
    audio = batch["audio"]

    # вычислим log-Mel входные признаки из входного аудио массива  
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # закодируем целевой текст в идентификаторы меток
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

Мы можем применить функцию подготовки данных ко всем нашим учебным примерам, используя метод `.map` в наборе данных. Аргумент `num_proc` указывает, сколько ядер процессора использовать. Если задать `num_proc` > 1, то будет включена многопроцессорная обработка. Если метод `.map` зависает при многопроцессорной обработке, установите `num_proc=1` и обрабатывайте набор данных последовательно.

> ВАЖНО!
> Для удобства работы я использую автоматическое определения количества рабочих прощессов через переменную `system_num_workers`. Это позволяет существенно повысить производительность и сэкономить время.

In [15]:
print("The current number of CPU cores is {}.".format(system_num_workers))
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc = system_num_workers)

The current number of CPU cores is 16.
                 

Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-47c4aa83b7ea6456.arrow


Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-0c9ad1f29fc71bd7.arrow


Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-b0819d7ebcae4c9d.arrow


Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-debd4b4c9fcd0638.arrow


...

Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-97030f8f4c45d3af.arrow


Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-2f492cb67c1917d5.arrow


Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-d3f5b610ed07fb40.arrow


Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-ddc3f812bcd3b5ad.arrow


Loading cached processed dataset at /home/artyom/.cache/huggingface/datasets/mozilla-foundation___common_voice_2_0/ru/2.0.0/c093ea99a0b8cadf95dd03e2ba81d28119744728ae97cc6196fd0c17b9b7f079/cache-9fe6a5c7e84c3487.arrow


## Обучение и оценка

Теперь, когда мы подготовили наши данные, мы готовы погрузиться в конвейер обучения. [🤗Тренер (Trainer)](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) сделает за нас большую часть тяжелой работы. Все, что нам нужно сделать, это:
- Определить коллатор данных (data collator): коллатор данных берет наши предварительно обработанные данные и подготавливает тензоры PyTorch, готовые для модели.
- Выбрать метрику для оценки: во время оценки мы хотим оценить модель с помощью метрики [word error rate (WER)](https://huggingface.co/metrics/wer). Нам нужно определить функцию `compute_metrics`, которая будет обрабатывать эти вычисления.
- Загрузить предварительно обученную контрольную точку: нам нужно загрузить предварительно обученную контрольную точку и правильно настроить ее для обучения.
- Определить конфигурацию процесса обучения: она будет использоваться 🤗 тренером для определения параметров тренировок.

После того как мы произведем тонкую настройку модели, мы оценим ее на тестовых данных, чтобы убедиться, что мы правильно обучили ее транскрибировать речь.

### Определение коллатора данных

Коллатор данных для модели последовательной речи уникален в том смысле, что он обрабатывает входные признаки `input_features` и метки `labels` независимо друг от друга: входные признаки `input_features` должны быть обработанны экстрактором признаков, а метки `labels` - токенизатором. 

Входные признаки `input_features` уже разбиты на аудиофрагменты длительностью 30 секунд и преобразованы в соответствующие им спектрограммы log-Mel фиксированной размерности с помощью экстрактора признаков, поэтому все, что нам нужно сделать, это преобразовать входные признаки `input_features` в тензоры PyTorch. Мы делаем это с помощью метода `.pad` экстрактора признаков с хаданием параметра `return_tensors=pt`.

С другой стороны, `метки (labels)` не имеют вставок (un-padded). Сначала мы выравниваем последовательности до максимальной длины в батче, используя метод `.pad` токенизатора. Затем токены-вставки заменяются на значение `-100`, чтобы эти токены **не** учитывались при вычислении потерь. Затем мы вырезаем токен BOS из начала последовательности меток, так как мы добавим его позже во время обучения. 

Мы можем использовать препроцессор `WhisperProcessor`, который мы определили ранее, для выполнения извлечения признаков, так и для работы с токенами:

In [16]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Давайте инициализируем коллатор данных, который мы только что определили:

In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Метрика оценки

Мы будем использовать метрику коэффициента ошибок в словах (WER), которая является "де-факто" метрикой для оценки систем ASR-систем. За дополнительной информацией обратитесь к документу WER [docs](https://huggingface.co/metrics/wer). Мы загрузим метрику WER из 🤗 Evaluate:

In [18]:
metric = evaluate.load("wer")

print("Metric is loaded successfully.")

Metric is loaded successfully.


Затем нам просто нужно определить функцию, которая принимает предсказания нашей модели и возвращает метрику WER. Эта функция, называемая `compute_metrics`, сначала заменяет `-100` на `pad_token_id` в `label_ids` (отменяя шаг, который мы применили в 
в коллаторе данных, чтобы правильно игнорировать подстановочные токены в потерях). Затем она декодирует прогнозы и идентификаторы меток в строки. Наконец, она вычисляет WER между прогнозами (predictions) и эталонными метками (reference labels):

In [19]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # заменяем -100 на pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # мы не хотим группировать токены при вычислении метрики
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

print("The compute_metrics function was defined successfully.")

The compute_metrics function was defined successfully.


### Загрузка контрольной точки предварительно обученной модели

Теперь давайте загрузим предварительно обученную контрольную точку модели Whisper `small`. Опять же, это тривиально благодаря использованию библиотеки 🤗 Transformers!

In [20]:
print("Current reference model:", reference_model)
model = WhisperForConditionalGeneration.from_pretrained(reference_model)

print("Checkpoint of the pre-trained model has been successfully loaded.")

Current reference model: openai/whisper-tiny
Checkpoint of the pre-trained model has been successfully loaded.


Переопределение аргументов генерации - никакие токены не выдаются принудительно в качестве выходов декодера (смотрите [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), никакие токены не подавляются во время генерации (дополнительно смотрите [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [21]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Определение конфигурации обучения

На последнем этапе мы определяем все параметры, связанные с процессом обучения. Более подробную информацию об аргументах процесса обучения можно найти в разделе Seq2SeqTrainingArguments [документации](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

> **Примечание**: если вы не хотите загружать контрольные точки модели в хаб HunggingFace, установите параметр `push_to_hub=False`.

> **ПРИМЕЧАНИЕ**:
> Здесь я разделяю настройки тренера для модели на три варианта:
> - Локальная среда, используется для обучения моделей на локальном ПК (),
> - Google Colab, так как в нем доступны GPU с большим обьемом VRAM бесплатно,
> - Kaggle, аналогично Google Colab доступны GPU с большим обьемом VRAM бесплатно.
>
> ***Оригиниальные настройки обучения модели сохранены в разделе Google Colab!***

#### Локальная среда

In [22]:
training_args = Seq2SeqTrainingArguments(
    output_dir = result_model_name,      # измените имя репозитория по своему усмотрению
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,       # увеличивается в 2x раза при каждом уменьшении размера батча в 2x раза  
    learning_rate=1e-6,
    warmup_steps=250,
    max_steps=50000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,        # Самый простоя способ задать равным параметру per_device_train_batch_size
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

#### Google Colab

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",       # измените имя репозитория по своему усмотрению
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,         # увеличивается в 2x раза при каждом уменьшении размера батча в 2x раза
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,          # Самый простоя способ задать равным параметру per_device_train_batch_size
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

#### Kaggle

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = result_model_name,        # измените имя репозитория по своему усмотрению
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,         # увеличивается в 2x раза при каждом уменьшении размера батча в 2x раза  
    learning_rate=1e-5,
    warmup_steps=250,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,          # Самый простоя способ задать равным параметру per_device_train_batch_size
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

### Задание настроек процесса обучения

Теперь мы можем передать аргументы обучения в 🤗 Trainer вместе с нашей моделью, набором данных, коллатором данных и функцией `compute_metrics`:

In [23]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/home/artyom/Whisper_Train/whisper-tiny-fine_tuned-ru is already a clone of https://huggingface.co/ElectricSecretAgent/whisper-tiny-fine_tuned-ru. Make sure you pull the latest changes with `repo.git_pull()`.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


### Обучение

Обучение займет примерно 5-10 часов в зависимости от вашего GPU или того, какой GPU был выделен для этого Google Colab. Если вы используете Google Colab непосредственно для тонкой настройки модели Whisper, вы должны убедиться, что обучение не будет прервано из-за бездействия. 
Простым обходным решением для предотвращения этого является вставить следующий код в консоль этой вкладки (_нажать правую кнопку мыши_ -> _обзор_ -> вкладка _консоль_ -> _вставить код_).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton, 60000);
```

Пиковое потребление памяти GPU для данной конфигурации обучения составляет примерно 15,8 ГБ. 
В зависимости от GPU, выделенного в Google Colab, возможно, что при запуске обучения вы столкнетесь с ошибкой CUDA `"out-of-memory"`. 
В этом случае вы можете уменьшить `per_device_train_batch_size` постепенно в 2 раза и использовать [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
для компенсации.

Чтобы запустить обучение, просто выполните:

In [ ]:
trainer.train()

/home/artyom/miniconda3/envs/pytorch-1.13.1/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3898
  Num Epochs = 206
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 50000
  Number of trainable parameters = 37760640
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use

Step,Training Loss,Validation Loss,Wer
500,0.365000,0.606537,199.271255
1000,0.138800,0.593255,188.965771
1500,0.060200,0.603169,59.182922
2000,0.024800,0.631755,89.760766
2500,0.012000,0.678006,164.497608
3000,0.005000,0.710920,103.400810
3500,0.003200,0.735493,97.563489
4000,0.002200,0.763451,102.002208
4500,0.001700,0.784493,113.816710
5000,0.001200,0.805583,100.331248


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

Наш лучший WER составляет 32.0% - неплохо для 8 часов обучения! Мы можем отправить нашу контрольную точку в [`hf-speech-bench`](https://huggingface.co/spaces/huggingface/hf-speech-bench) на push, задав соответствующие аргументы ключевых слов (kwargs):

In [38]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_2_0",
    "dataset": "Common Voice 2.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ru, split: test",
    "language": "ru",
    "model_name": "Whisper Tiny - Boyko Artyom",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

Теперь результаты обучения можно загрузить в хаб. Для этого выполните команду `push_to_hub` и сохраните созданный нами объект препроцессора:

In [39]:
trainer.push_to_hub(**kwargs)

Saving model checkpoint to whisper-tiny-fine_tuned-ru
Configuration saved in whisper-tiny-fine_tuned-ru/config.json
Model weights saved in whisper-tiny-fine_tuned-ru/pytorch_model.bin
Feature extractor saved in whisper-tiny-fine_tuned-ru/preprocessor_config.json


TypeError: transformers.trainer.Trainer.create_model_card() got multiple values for keyword argument 'model_name'

## Загрузка токенизатора в хаб

Сохраним используемый токенизатор в хаб:

In [26]:
tokenizer.push_to_hub(result_model_name)

tokenizer config file saved in whisper-tiny-fine_tuned-ru/tokenizer_config.json
Special tokens file saved in whisper-tiny-fine_tuned-ru/special_tokens_map.json
added tokens file saved in whisper-tiny-fine_tuned-ru/added_tokens.json
Uploading the following files to ElectricSecretAgent/whisper-tiny-fine_tuned-ru: added_tokens.json,tokenizer_config.json,merges.txt,vocab.json,normalizer.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/ElectricSecretAgent/whisper-tiny-fine_tuned-ru/commit/439ee2e80baae35cb509d881fe4857780e807218', commit_message='Upload tokenizer', commit_description='', oid='439ee2e80baae35cb509d881fe4857780e807218', pr_url=None, pr_revision=None, pr_num=None)

## Создание демонстрации

Теперь, когда мы доработали нашу модель, мы можем создать демонстрационный образец, чтобы продемонстрировать ее возможности ASR! 
Мы будем использовать конвейере 🤗 Transformers `pipeline`, который позаботится обо всем, начиная с предварительной обработки аудиовходов и заканчивая декодированием предсказаний модели.

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="sanchit-gandhi/whisper-small-hi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

## Заключительные замечания

В этом блоге мы рассмотрели пошаговое руководство по тонкой настройке Whisper для многоязыкового ASR используя 🤗 Datasets, Transformers и хаб Hugging Face. 
Для получения более подробной информации о модели Whisper, наборе данных Common Voice и теории, лежащей в основе тонкой настройки, обратитесь к сопроводительной записи [blog post](https://huggingface.co/blog/fine-tune-whisper).
Если вы заинтересованы в тонкой настройке других моделей Transformers, как для английского, так и для многоязычного ASR, обязательно ознакомьтесь с примерами скриптов в [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).

# Полезные ссылки

- [Fine-Tune Whisper For Multilingual ASR with Transformers (Статья)](https://huggingface.co/blog/fine-tune-whisper)